In [1]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Conv2D, Dense, Flatten, Input
from keras.layers.merge import Concatenate
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = open("body.txt", "r")
tdata = np.empty((0,48), dtype=float)
tlabel = []
train_images = []

for d in data.readlines():
    tdata = np.array([float(int(t)) for t in d.split(" ")[24:48]]).reshape((1,24))
    tlabel.append(int(d.split(" ")[48].strip("\n")))
    tdata = tdata.reshape(1,6,4)
    max0 = abs(np.max(tdata[0]))

    if (max0 != 0):
        tdata[0] = tdata[0] / max0
    
    train_images.append(tdata)

train_images = np.array(train_images)
tlabel = np.array(to_categorical(tlabel))
print(train_images.shape)
print(tlabel.shape)

(254, 1, 6, 4)
(254, 3)


In [3]:
train_images = np.moveaxis(train_images, [1,2,3], [3,1,2])
print(train_images.shape)

(254, 6, 4, 1)


In [4]:
x_train, x_test, y_train, y_test = train_test_split(train_images, tlabel, test_size=0.05)

In [5]:
num_filters = 1
filter_size = 2
initializer = keras.initializers.RandomNormal(mean=0., stddev=1.)

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(6,4,1), kernel_initializer=initializer),
  Conv2D(num_filters, filter_size, input_shape=(5,3), kernel_initializer=initializer),
  Flatten(),
  Dense(16, activation='sigmoid', kernel_initializer=initializer),
  Dense(8, activation='sigmoid', kernel_initializer=initializer),
  Dense(4, activation='sigmoid', kernel_initializer=initializer),
  Dense(3, activation='softmax', kernel_initializer=initializer)
])

In [6]:
opt = keras.optimizers.Nadam(
  learning_rate=0.001,
  beta_1=0.9,
  beta_2=0.999,
  epsilon=1e-07
  )

model.compile(
  loss='binary_crossentropy',
  optimizer=opt,
  metrics=['accuracy'],
)

In [14]:
model.fit(
    x_train,
    y_train,
    epochs=1000,
    validation_data=(x_test, y_test)
    )

Train on 241 samples, validate on 13 samples
Epoch 1/1000
241/241 [==============================] - 1s 3ms/step - loss: 0.5634 - accuracy: 0.6999 - val_loss: 0.5091 - val_accuracy: 0.7692
Epoch 2/1000
241/241 [==============================] - 0s 251us/step - loss: 0.5604 - accuracy: 0.7082 - val_loss: 0.5053 - val_accuracy: 0.7692
Epoch 3/1000
241/241 [==============================] - 0s 214us/step - loss: 0.5571 - accuracy: 0.7123 - val_loss: 0.5014 - val_accuracy: 0.7692
Epoch 4/1000
241/241 [==============================] - 0s 213us/step - loss: 0.5541 - accuracy: 0.7082 - val_loss: 0.4976 - val_accuracy: 0.7692
Epoch 5/1000
241/241 [==============================] - 0s 211us/step - loss: 0.5509 - accuracy: 0.7151 - val_loss: 0.4941 - val_accuracy: 0.7692
Epoch 6/1000
241/241 [==============================] - 0s 208us/step - loss: 0.5479 - accuracy: 0.7178 - val_loss: 0.4909 - val_accuracy: 0.7692
Epoch 7/1000
241/241 [==============================] - 0s 219us/step - loss: 0.5

In [15]:
model.save_weights('body.h5')

In [7]:
model.load_weights('body.h5')

In [9]:
weights = np.array(model.get_weights())
print(weights)

[array([[[[-2.3075743]],

        [[ 0.6278243]]],


       [[[-0.5512385]],

        [[-1.751918 ]]]], dtype=float32)
 array([0.2116318], dtype=float32)
 array([[[[ 1.4289988]],

        [[ 3.4592788]]],


       [[[-0.6329475]],

        [[ 2.0713494]]]], dtype=float32)
 array([0.1789779], dtype=float32)
 array([[-2.0937805 ,  0.91976655, -0.58942246, -1.7812543 ,  1.1439905 ,
         0.16461086,  1.3675767 , -1.3383008 , -0.74018353,  2.6037    ,
        -0.92381155,  0.51548815,  0.6687302 , -0.34654072,  0.3627134 ,
        -2.0461082 ],
       [-0.01297659,  0.1307955 ,  0.23839527, -0.21899943,  0.44152635,
         0.29893634,  0.9848455 , -0.58062184, -0.97889125,  0.97074616,
         2.049569  , -2.339076  ,  1.5523267 ,  0.90956086, -1.0569621 ,
        -0.18869935],
       [ 1.261815  , -2.228345  , -0.10889169, -1.2571992 ,  0.56670934,
         0.5368318 , -1.0960104 ,  1.9485408 , -1.0275377 , -0.07301141,
        -0.7944158 ,  0.2556189 , -0.660745  ,  0.4933999 , -0.

In [17]:
weight_file = open("bodyOutput.txt", "w")

for w in weights[0:]:
    weight_file.write("{")

    if(len(w.shape)==2):
        for i in np.array(w):
            weight_file.write("{")
            i=np.array(i)

            for j in np.array(i):
                weight_file.write(str(j) + ", ")
            
            weight_file.write("}, \n")
    
    else:
        for j in np.array(w):
            weight_file.write(str(j) + ", ")

    weight_file.write("}\n\n")

weight_file.flush()
weight_file.close()

In [59]:
testInput = np.array([[[0.0,0.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,1.0,0.0],
                        [0.0,1.0,1.0,0.0],[0.0,1.0,1.0,0.0],[0.0,1.0,1.0,0.0]],
                      [[957.0,949.0,958.0,958.0],[952.0,951.0,958.0,958.0],[952.0,949.0,947.0,947.0],
                      [952.0,962.0,962.0,962.0],[952.0,962.0,962.0,962.0],[952.0,962.0,962.0,962.0]]]).reshape(1,1,6,4)

ValueError: cannot reshape array of size 48 into shape (1,1,6,4)

In [58]:
max0 = abs(np.max(testInput[0,0]))

if (max0 != 0):
    testInput[0,0] = testInput[0,0] / max0

In [14]:
testInput = np.moveaxis(testInput, [1,2,3], [3,1,2])
model.predict(testInput)

array([[0.84017104, 0.07625445, 0.0835745 ]], dtype=float32)

In [15]:
outputs = [K.function([model.input], [layer.output])([testInput,1]) for layer in model.layers]
print(np.array(outputs[0]).shape)

(1, 1, 3, 2, 1)


In [9]:
data = open("weight.txt", "r")
tdata = []
tlabel = np.empty((0,1), dtype=float)
bodyOutput = []
train_images = []

for d in data.readlines():
    temp = np.array([float(t) for t in d.split(" ")[24:49]]).reshape((1,25))
    tdata = temp[0][:24]
    bodyOutput.append(temp[0][24])
    tlabel = np.append(tlabel, np.array([float(d.split(" ")[49].strip("\n"))]).reshape(1,1), axis=0)
    tdata = tdata.reshape(1,6,4)
    max0 = abs(np.max(tdata[0]))

    if (max0 != 0):
        tdata[0] = tdata[0] / max0
    
    train_images.append(tdata)

train_images = np.array(train_images)
bodyOutput = np.array(bodyOutput)
bodyOutput = bodyOutput.reshape(bodyOutput.shape[0],1)
print(train_images.shape)
print(bodyOutput.shape)
print(tlabel.shape)

(195, 1, 6, 4)
(195, 1)
(195, 1)


In [10]:
train_images = np.moveaxis(train_images, [1,2,3], [3,1,2])
print(train_images.shape)

(195, 6, 4, 1)


In [11]:
shufflePoints = np.random.permutation(np.arange(0, train_images.shape[0], 1))
trainingPoints = shufflePoints[0:int(train_images.shape[0]*0.95)]
validationPoints = shufflePoints[int(train_images.shape[0]*0.95):]

x_train = train_images[trainingPoints]
x_test = train_images[validationPoints]
y_train = tlabel[trainingPoints]
y_test = tlabel[validationPoints]
bodyOutput_train = bodyOutput[trainingPoints]
bodyOutput_test = bodyOutput[validationPoints]

print(x_train.shape)
print(x_test.shape)
print(bodyOutput_train.shape)
print(bodyOutput_test.shape)
print(y_train.shape)
print(y_test.shape)

(185, 6, 4, 1)
(10, 6, 4, 1)
(185, 1)
(10, 1)
(185, 1)
(10, 1)


In [19]:
num_filters = 1
filter_size = 2
initializer = keras.initializers.RandomNormal(mean=0., stddev=1.)

sequence_input = Input(batch_shape=x_train.shape, dtype=float, name='text_input')
model = (Conv2D(filters=num_filters, kernel_size=filter_size, input_shape=(6,4,1), kernel_initializer=initializer))(sequence_input)
model = (Conv2D(filters=num_filters, kernel_size=filter_size, input_shape=(5,3), kernel_initializer=initializer))(model)
model = (Flatten())(model)
af_input = Input(batch_shape=(bodyOutput_train.shape), name='af_input')
model = Concatenate()([model, af_input])
model = (Dense(16, activation='relu', kernel_initializer=initializer))(model)
model = (Dense(8, activation='relu', kernel_initializer=initializer))(model)
model = (Dense(4, activation='relu', kernel_initializer=initializer))(model)
model = (Dense(2, activation='relu', kernel_initializer=initializer))(model)
model = (Dense(1, activation='relu', kernel_initializer=initializer))(model)
model = Model(inputs=[sequence_input, af_input], outputs=model)

In [20]:
opt = keras.optimizers.Adamax(
    learning_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
    )

model.compile(
    loss="mean_absolute_error",
    optimizer=opt
    )

TypeError: Unexpected keyword argument passed to optimizer: beta_1

In [21]:
model.fit(
    [x_train, bodyOutput_train],
    y_train,
    batch_size=185,
    epochs=2500,
    validation_data=([x_test, bodyOutput_test], y_test)
    )

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [15]:
model.save_weights('weight.h5')

In [16]:
model.load_weights('weight.h5')

In [17]:
weights = np.array(model.get_weights())
print(weights)

[array([[[[ 0.5031037 ]],

        [[-1.1943668 ]]],


       [[[ 1.1884304 ]],

        [[ 0.15919141]]]], dtype=float32)
 array([0.03181743], dtype=float32)
 array([[[[ 0.980233  ]],

        [[-2.0326009 ]]],


       [[[ 0.15641993]],

        [[ 0.1293441 ]]]], dtype=float32)
 array([-0.04224933], dtype=float32)
 array([[ 0.02811451,  0.14115074, -0.7596182 , -0.45216173,  1.9632068 ,
        -1.1361525 ,  2.0007827 , -0.60919017, -1.4843904 , -0.70890886,
        -1.1409081 , -3.9168832 ,  2.0058393 , -3.834855  , -0.94839835,
        -0.30564082],
       [-0.88946605, -0.20174566, -1.1342117 ,  0.7685776 , -0.9991824 ,
        -0.27439615,  0.9679049 ,  1.5804557 ,  0.58227915, -0.26640683,
         2.1141198 , -1.5843858 ,  0.5486696 ,  0.21486801,  0.5285858 ,
         0.31330842],
       [ 1.6950445 ,  1.2089963 , -0.2690099 ,  0.56573206, -0.31037828,
         0.6945764 , -0.3902347 , -2.3587089 ,  0.5282625 ,  0.37267423,
         2.1613967 , -2.199097  ,  1.8299128 ,  1.20

In [18]:
weight_file = open("weightOutput.txt", "w")

for w in weights[0:]:
    weight_file.write("{")

    if(len(w.shape)==2):
        for i in np.array(w):
            weight_file.write("{")
            i=np.array(i)

            for j in np.array(i):
                weight_file.write(str(j) + ", ")
            
            weight_file.write("}, \n")
    
    else:
        for j in np.array(w):
            weight_file.write(str(j) + ", ")
    
    weight_file.write("}\n\n")

weight_file.flush()
weight_file.close()

In [ ]:
testInput = np.array([[[0.0,0.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,1.0,0.0],
                        [0.0,1.0,1.0,0.0],[0.0,1.0,1.0,0.0],[0.0,1.0,1.0,0.0]],
                      [[957.0,949.0,958.0,958.0],[952.0,951.0,958.0,958.0],[952.0,949.0,947.0,947.0],
                      [952.0,962.0,962.0,962.0],[952.0,962.0,962.0,962.0],[952.0,962.0,962.0,962.0]]]).reshape(1,1,6,4)

In [27]:
max0 = abs(np.max(testInput[0,0]))
if (max0 != 0):
    testInput[0,0] = testInput[0,0] / max0

In [28]:
testInput = np.moveaxis(testInput, [1,2,3], [3,1,2])
testBodyInput = np.array([1.]).reshape(1,1)
model.predict([testInput, testBodyInput])

array([[65.2497]], dtype=float32)

In [29]:
outputs = [K.function([model.input], [layer.output])([[testInput, testBodyInput], 1]) for layer in model.layers]
print(outputs)

[[array([[[[ 0.02739726,  0.9746707 ],
         [ 0.42465752,  0.9280648 ],
         [ 0.15068494,  0.970618  ]],

        [[ 0.06849315,  0.9554205 ],
         [ 1.        ,  0.88247216],
         [ 0.16438356,  0.964539  ]],

        [[-0.5479452 ,  1.        ],
         [ 0.1780822 ,  0.9544073 ],
         [ 0.06849315,  0.95947313]],

        [[ 0.01369863,  0.970618  ],
         [ 0.08219178,  0.9625127 ],
         [ 0.02739726,  0.97771025]]]], dtype=float32)], [array([[[[-0.49334103],
         [-0.47290057]],

        [[-0.5350699 ],
         [-0.5495598 ]],

        [[-0.55490917],
         [-0.5610505 ]]]], dtype=float32)], [array([[-0.49334103, -0.47290057, -0.5350699 , -0.5495598 , -0.55490917,
        -0.5610505 ]], dtype=float32)], [array([[1.]], dtype=float32)], [array([[-0.49334103, -0.47290057, -0.5350699 , -0.5495598 , -0.55490917,
        -0.5610505 ,  1.        ]], dtype=float32)], [array([[-0.2861442 , -0.5914042 , -0.56892747, -0.5486933 ,  1.0697259 ,
         0.9